# Lab3 Intro
<span style="color: red;">This step requires a high RPM (Request Per Minitues), exceeding the the workshop environment' limit. You're scheduled for a reading session, but you can still run it in your own AWS account</span>
### Using RAG technique to enhance the solution by retrieving and injecting few shot examples into the prompt
### What is RAG?
RAG is a technique for augmenting LLM (Large Language Model) knowledge with additional data.
Large Language Models (LLMs) can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

A typical RAG application has two main components:

- **Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happens offline.

- **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

### What is Few-Shot Learning?
Few-shot learning is a machine learning paradigm that enables models to generalize from a limited number of labeled examples. Unlike traditional supervised learning, which requires large datasets for training, few-shot learning allows models to adapt to new tasks or classes with minimal data. This is akin to how humans can learn new concepts quickly from just a few instances.

### Using RAG and Few-Shot Learning to Classify VoC
With the introduction of RAG and Few-Shot Learning, we can now apply these concepts practically. In this lab, we will utilize the RAG technique together with Few-Shot Learning to classify Voice of Customer data. This session will build upon the skills developed in the last two labs, as we will continue to employ prompt engineering and embedding techniques. 

### Your objectives are:
- Setup a vector database using [Chroma integration in LangChain](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/) 
- Use Amazon Titan embedding model to embed the labeled examples and store into the vector database
- Build a RAG chain that retrieves top k most relevant examples from vector store and generate the classification result

## 1. Install dependencies
- If you experince "ERROR: pip's dependency resolver does not currently...", please just ignore it

In [9]:
!pip install -q langchain==0.2.16 langchain_aws==0.1.17 pandas==2.2.2 openpyxl==3.1.5 chromadb==0.5.5 langchain-chroma==0.1.2 python-dotenv==1.0.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.233.0 requires protobuf<5.0,>=3.12, but you have protobuf 5.29.0 which is incompatible.


## 2. Initialize Bedrock model using Langchain

We will utilize the same approach used in Lab2 to setup the embedding object. 
- We use [Langchain](https://www.langchain.com/) SDK to build the application
- Initialize a BedrockEmbeddings object with 'Titan Text Embeddings V2" with the model id "amazon.titan-embed-text-v2:0"

In [2]:
import boto3
import json
import copy
import pandas as pd
from termcolor import colored
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
bedrock_embedding = BedrockEmbeddings(model_id='amazon.titan-embed-text-v2:0')

- test run and preview the result

In [3]:
test_embedding = bedrock_embedding.embed_documents(['I love programing'])
print(f"The embedding dimension is {len(test_embedding[0])}, first 10 elements are: {test_embedding[0][:10]}")

The embedding dimension is 1024, first 10 elements are: [-0.0879908874630928, 0.06109211966395378, -0.03054605983197689, 0.023365456610918045, 0.08297586441040039, 0.009916071780025959, 0.014931096695363522, -0.036700863391160965, 0.020402032881975174, -0.04376748949289322]


## 3. Preparing the Dataset to be classified. 
- The data (Voice of Customers) is subject to experiment usage only

We will once again load the categories.csv and comments.csv data files. In addition, we will introduce a new dataset, examples_with_label.csv, in this session. This new dataset provides an augmented knowledge base for the LLM, equipping it with relevant references. By incorporating these labeled examples into our workflow, it is expected to enhance the model's understanding and improve the accuracy of its output.

- Import libraries

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
category_definition = "data/categories.csv"
categories = pd.read_csv(category_definition)
display(categories)

,mappings
0,Camera lens glare
1,CarPlay connection failure
2,Update delay
3,Camera dust and dirt
4,"Automatic restart, shutdown"
5,High storage usage
6,Poor manufacturing quality
7,Frame rate drop
8,Camera black screen
9,App crashes


### Load the customer review data

In [6]:
comments_filepath = "data/comments.csv"
comments = pd.read_csv(comments_filepath)
comments

,comment,groundtruth
0,"I was almost done with the update, the progres...",Update failure
1,"Scratches, peeling paint, dust in the lens... ...",Poor manufacturing quality
2,"My phone has been having issues lately, freque...",Device freezing
3,"After the update, WeChat often crashes and som...",App crashes
4,The connectivity issues with this phone are re...,CarPlay connection failure
...,...,...
95,"After the recent software update, my device ha...","Automatic restart, shutdown"
96,Another bug in the new smartphone model. It se...,Activation failure
97,"This phone just doesn't feel right in my hand,...",Poor manufacturing quality
98,"After upgrading yesterday, I've started experi...",Weak signal


### Load examples data as we will use them as few shot examples

In [7]:
examples_filepath = "data/examples_with_label.csv"
examples_df = pd.read_csv(examples_filepath) 
examples_df

,comment,groundtruth
0,This phone's camera really isn't that great. T...,Camera color deviation
1,My phone has been having some issues lately. T...,Screen flickering
2,My phone has been experiencing some subtle iss...,CarPlay connection failure
3,My phone has been a real headache lately. The ...,Weak signal
4,My phone has been having some issues with the ...,Screen color deviation
...,...,...
184,"This phone is so unreliable, it keeps disappea...","Automatic restart, shutdown"
185,"Since the latest software update, my phone has...",Wi-Fi connection failure
186,"I've been using this phone for a while now, an...",Device freezing
187,My phone has been draining its battery really ...,Fast battery drain


## 4. Create Vector database to store the embedding vectors of customer review text

### 4.1 We will setup a vector database using [Chroma integration in LangChain](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/) 
- Chroma is the AI-native open-source vector database. This is lightweight vector database suitable for developers to quickly develop and experiment with applications.
- We will use Chroma for our workshop experiment this time, but when you will develop your own application, there are variety of selections, such as Amazon OpenSearch, Amazon Aurora, Pinecone, Milvus,Faiss, etc. 

In [10]:
from langchain_chroma import Chroma
# You can uncomment below code to reset the vector db, if you want to retry more times
# vector_store.reset_collection()  
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=bedrock_embedding,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

#### Let's consider each customer comment as an individual document, and then use the Langchain-Chroma SDK to upload all these customer comments into the vector store.

In [11]:
from uuid import uuid4
import hashlib
from langchain_core.documents import Document

#### build langchain documents

In [12]:
documents = []
for comment,groundtruth in examples_df.values:
    documents.append(
        Document(
        page_content=comment,
        metadata={"groundtruth":groundtruth}
        )
    )

#### add documents to vector store

In [13]:
hash_ids = [hashlib.md5(doc.page_content.encode()).hexdigest() for doc in documents]
vector_store.add_documents(documents=documents, ids=hash_ids)

['3b5fabfbd3d9c083d5e7655fcd59e9e0',
 '024e991ccfa2538dad706911ded85fe7',
 '624aaa6f69c5c2926be806573cb2448c',
 '42ca012f3765fb4e9f391b6a9cd71e57',
 'ff871bc5ef7501e04c63f29caaf285e9',
 '30dd9e119138a66c4f0b34cb2fd7a4df',
 '10006bc5d8c0cbf831a0ffd83b8af854',
 '9f6e78cd3b035cf80dfa10c61e3fd7c9',
 'dd8fde4eeebb3b675872f85d42f0eb95',
 '1527eacb9589e6fa9461aef47beb2158',
 'abc164b7cc3f9c1e0d207b822fe2d2db',
 '722438ba64654367a3537ba2889c0a7d',
 '70bba6fa8c42bbd6199b56cba038308c',
 '2a827f3b9707fada0c77452350550c21',
 '7523b3229ae2ccb5fab3dbbcf662afd8',
 'cb113aafe6e94104f5ecb528c16432c8',
 '289fd4075e9a7f13493cd7cd05d17707',
 'fa90ece2842518950e3dd423aca958c1',
 'b45c9e2f99f07e23d54519a97d48440b',
 'e83910e172516897c5fabc278feb675e',
 '7d374351eaa37a7d0240463a32d4f04c',
 '6a676fe6b2f0ea478b7ccf1e62e2f0df',
 'bd04dc11853f529b1d2bada3140fc43e',
 'c6d85ae44f3542da57a63b07904c7ad7',
 '15e61456a4e187ee11e5da49c3f13372',
 '4a11f610128d9fef006029de92713e9c',
 'ba0f900f8d87a178f534f96a23fb766b',
 


### 4.2 We will use vector search to compare the similarity between them and choose the category that is the similarest to the customer review text

Once the embedded customer review texts are stored in the vector database, we can apply the techniques we explored in Lab 2, semantic similarity search, to identify top k semantically relevant categorized review texts for each customer comment. 

- Test run Similarity search in vector store
- Performing a simple similarity search with relevance score can be done as follows:

In [14]:
query = comments['comment'].sample(1).values[0]
print(colored(f"******query*****:\n{query}","blue"))

results = vector_store.similarity_search_with_relevance_scores(query, k=4)
print(colored("\n\n******results*****","green"))
for res, score in results:
    print(colored(f"* [SIM={score:3f}] \n{res.page_content}\n{res.metadata}","green"))

******query*****:
The new desktop layout is so beautiful! It's so advanced! Aside from the weak signal, the rest of the phone is really great!!! Would you just throw it in the trash because of the weak signal?


******results*****
* [SIM=0.061232] 
My phone signal is really terrible, it keeps cutting in and out. Browsing the web and making calls is not very stable, and it sometimes even just disconnects completely. It's really annoying, how can a phone this nice have this kind of issue? I hope the manufacturer can resolve this problem quickly, otherwise I might have to get a new phone.
{'groundtruth': 'Weak signal'}
* [SIM=0.053585] 
I've been using the new smartphone model for a while now, and while it has some great features, I've also encountered a few issues. Some of these problems are from my own experience, while others have been reported by other users.

1. Camera issues
Occasionally, there are instances where the camera produces blotchy or distorted images. This seems to be a s

## 5. Build a RAG chain to retrieve only top K relevant examples for classification

#### 5.1 Customzie a LangChain Chat Model Class
- As the time of this event, LangChain has not supported the latest Amazon Foundation Model yet, we will customize a LangChain Chat Model Class, so that the latest model can be integrated with the chain prompting in LangChain.

In [15]:
import boto3
import json
from botocore.exceptions import ClientError
import dotenv
import os
dotenv.load_dotenv()

from typing import Any, AsyncIterator, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    AsyncCallbackManagerForLLMRun,
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel, SimpleChatModel
from langchain_core.messages import AIMessageChunk, BaseMessage, HumanMessage,AIMessage,SystemMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from langchain_core.runnables import run_in_executor
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser,XMLOutputParser
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate


class ChatModelNova(BaseChatModel):

    model_name: str
    br_runtime : Any = None
    ak: str = None
    sk: str = None
    region:str = None

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:

        if not self.br_runtime:
            if self.ak and self.sk:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime',
                                               region_name = self.region,
                                              aws_access_key_id = self.ak,
                                               aws_secret_access_key = self.sk
            
                                              )

            else:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime')
            
        
        new_messages = []
        system_message = ''
        for msg in messages:
            if isinstance(msg,SystemMessage):
                system_message = msg.content
            elif isinstance(msg,HumanMessage):
                new_messages.append({
                        "role": "user",
                        "content": [ {"text": msg.content}]
                    })
            elif isinstance(msg,AIMessage):
                new_messages.append({
                        "role": "assistant",
                        "content": [ {"text": msg.content}]
                    })

        
        temperature = kwargs.get('temperature',0.5)
        maxTokens = kwargs.get('max_tokens',3000)

        #Base inference parameters to use.
        inference_config = {"temperature": temperature,"maxTokens":maxTokens}


        # Send the message.
        response = self.br_runtime.converse(
            modelId=self.model_name,
            messages=new_messages,
            system=[{"text" : system_message}] if system_message else [],
            inferenceConfig=inference_config
        )
        output_message = response['output']['message']

        message = AIMessage(
            content=output_message['content'][0]['text'],
            additional_kwargs={},  # Used to add additional payload (e.g., function calling request)
            response_metadata={  # Use for response metadata
                **response['usage']
            },
        )
        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])


    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        if not self.br_runtime:
            if self.ak and self.sk:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime',
                                               region_name = self.region,
                                              aws_access_key_id = self.ak,
                                               aws_secret_access_key = self.sk
            
                                              )

            else:
                self.br_runtime = boto3.client(service_name = 'bedrock-runtime')
            
        
        new_messages = []
        system_message = ''
        for msg in messages:
            if isinstance(msg,SystemMessage):
                system_message = msg.content
            elif isinstance(msg,HumanMessage):
                new_messages.append({
                        "role": "user",
                        "content": [ {"text": msg.content}]
                    })
            elif isinstance(msg,AIMessage):
                new_messages.append({
                        "role": "assistant",
                        "content": [ {"text": msg.content}]
                    })

        
        temperature = kwargs.get('temperature',0.5)
        maxTokens = kwargs.get('max_tokens',3000)

        #Base inference parameters to use.
        inference_config = {"temperature": temperature,"maxTokens":maxTokens}

        # Send the message.
        streaming_response = self.br_runtime.converse_stream(
            modelId=self.model_name,
            messages=new_messages,
            system=[{"text" : system_message}] if system_message else [],
            inferenceConfig=inference_config
        )
        # Extract and print the streamed response text in real-time.
        for event in streaming_response["stream"]:
            if "contentBlockDelta" in event:
                text = event["contentBlockDelta"]["delta"]["text"]
                # print(text, end="")
                chunk = ChatGenerationChunk(message=AIMessageChunk(content=[{"type":"text","text":text}]))

                if run_manager:
                    # This is optional in newer versions of LangChain
                    # The on_llm_new_token will be called automatically
                    run_manager.on_llm_new_token(text, chunk=chunk)

                yield chunk
            if 'metadata' in event:
                metadata = event['metadata']
                # Let's add some other information (e.g., response metadata)
                chunk = ChatGenerationChunk(
                    message=AIMessageChunk(content="", response_metadata={**metadata})
                )
                if run_manager:
                    run_manager.on_llm_new_token('', chunk=chunk)
                yield chunk

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "echoing-chat-model-advanced"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.

        This information is used by the LangChain callback system, which
        is used for tracing purposes make it possible to monitor LLMs.
        """
        return {
            "model_name": self.model_name,
        }

llm = ChatModelNova(model_name="amazon.nova-pro-v1:0")

- Define system prompt and user prompt template

The system prompt template has been copied over from Lab 1, while the user prompt has been modified slightly. For this user prompt, we add a part where we provide examples to the LLM. This is a demonstration for Few-Shot Learning. The model will takes these exmaples as references when making classifications.

In [16]:
system = """You are a professional  customer feedback analyst. Your daily task is to categorize user feedback.
You will be given an input in the form of a JSON array. Each object in the array contains a comment ID and a 'c' field representing the user's comment content.
Your role is to analyze these comments and categorize them appropriately.
Please note:
1. Only output valid XML format data.
2. Do not include any explanations or additional text outside the XML structure.
3. Ensure your categorization is accurate and consistent.
4. If you encounter any ambiguous cases, use your best judgment based on the context provided.
5. Maintain a professional and neutral tone in your categorizations.
"""

user = """
Please categorize user comments according to the following category tags library:
<categories>
{tags}
</categories>

Here are examples for your to categorize:
<examples>
{examples}
<examples>

Please follow these instructions for categorization:
<instruction>
1. Categorize each comment using the tags above. If no tags apply, output "Invalid Data".
2. Summarize the comment content in no more than 50 words. Replace any double quotation marks with single quotation marks.
</instruction>

Below are the customer comments records to be categorized. The input is an array, where each element has an 'id' field representing the complaint ID and a 'c' field summarizing the complaint content.
<comments>
{input}
</comments>

For each record, summarize the comment, categorize according to the category explainations, and return the  ID, summary , reasons for tag matches, and category.

Output format example:
<output>
  <item>
    <id>xxx</id>
    <summary>xxx</summary>
    <reason>xxx</reason>
    <category>xxx</category>
  </item>
</output>

Skip the preamble and output only valid XML format data. Remember:
- Avoid double quotation marks within quotation marks. Use single quotation marks instead.
- Replace any double quotation marks in the content with single quotation marks.
"""

#### 5.2 Define a RAG pipeline with LangChain

We are now set to create a RAG pipeline using Langchain. The workflow of this pipelines involves:
- initially, it retrieves the top k most relevant customer review texts, which are labeled with categories, from the vector database. 
- This retrieved information is then integrated into the prompt. 
- Invoke LLM to generate outputs according to the provided instructions. 
- Finally, the output is displayed. 

- Define langchain retriever, use `k = 5` to retrive top 5 records

In [17]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [18]:
def format_docs(docs):
    formatted = "\n".join([f"<content>{doc.page_content}</content>\n<cateogry>{doc.metadata['groundtruth']}</cateogry>" for doc in docs])
    # print(colored(f"*****retrived examples******:\n{formatted}","yellow"))
    return formatted

- test run for retriever

In [19]:
retrieved_docs = (retriever|format_docs).invoke("my phone always freeze")

- Define prompt template

In [20]:
prompt = ChatPromptTemplate([
    ('system',system),
    ('user',user)],
partial_variables={'tags':categories['mappings'].values}
)

- Define a RAG prompt chain

In [21]:
chain = (
    {"examples":retriever|format_docs,"input":RunnablePassthrough()}
    | prompt
    | llm
    | XMLOutputParser()
)

- Convert comments into a data array for iterative processing.

In [22]:
sample_data = [str({"id":'s_'+str(i),"comment":x[0]}) for i,x in enumerate(comments.values)]
print(sample_data[:3])

['{\'id\': \'s_0\', \'comment\': "I was almost done with the update, the progress bar had finished, and then suddenly the device just started spinning and displaying a blank screen. I\'m not sure what happened, but I\'m really disappointed that the update failed. [Sad emoji, Sad emoji, Sad emoji]"}', "{'id': 's_1', 'comment': 'Scratches, peeling paint, dust in the lens... The product quality seems to have gone downhill. #smartphone'}", '{\'id\': \'s_2\', \'comment\': "My phone has been having issues lately, frequently freezing up and becoming completely unresponsive. It\'s really been affecting the user experience. Sometimes it even restarts on its own before I have a chance to do anything, which is just baffling. I\'m not sure if it\'s a system problem, and I don\'t know how to fix it. It\'s feeling more and more difficult to use. I hope the manufacturer can address this problem quickly, otherwise I may have to consider getting a new phone."}']


### We will iterate through each comment for categorization.
* This step will take around 4~5 mins to compelete
* <span style="color: red;">Hint!</span> Due to the uncertainly of LLM generation, the JSON output might fail occasionally, if you happen to experience error such as `KeyError: 'output'`, please re-run the cell again

- Copy below code to a new cell and run if you are in your own account
```python
import math,json
from json import JSONDecodeError

resps = []


def retry_call(chain,args: Dict[str,Any],times:int=3):
    """
      Retry mechanism to ensure the success rate of final structure output 
    """
    try:
        content = chain.invoke(args)
        if 'output' not in content:
            raise (JSONDecodeError('KeyError: output'))
        return content
    except Exception as e:
        if times:
            print(f'Exception, return [{times}]')
            return retry_call(chain,args,times=times-1)
        else:
            raise(JSONDecodeError(e))
if cmd != 'confirm':
    for i in range(len(sample_data)):
        data = sample_data[i]
        # print(colored(f"*****input[{i}]*****:\n{data}","blue"))

        # resp = chain.invoke(data)
        resp = retry_call(chain,data)
        print(colored(f"*****response*****\n{resp}","green"))
        # resps += json.loads(resp)
        for item in resp['output']:
            row={}
            for it in item['item']:
                row[list(it.keys())[0]]=list(it.values())[0]
            resps.append(row)

```

In [ ]:
#placeholder: if you are in your own account, copy the example code above and past it here to run

### Check if all the data has been processed correctly

In [ ]:
assert len(resps) == len(sample_data), "Due to the uncertainly of LLM generation, the JSON output might fail occasionally, if you happen to experience error, please re-run above cell again"

- covert the data array to pandas dataframe

In [ ]:
prediction_df = pd.DataFrame(resps).rename(columns={"category":"predict_label"}).drop_duplicates(['id']).reset_index(drop='index')
# convert the label value to lowercase
prediction_df['predict_label'] = prediction_df['predict_label'].apply(lambda x: x.strip().lower().replace("'",""))
prediction_df

### Merge the prediction result to ground truth dataframe

- copy comments to ground_truth dataframe
- merge the date prediction to the groudtruth data

In [24]:
ground_truth = comments.copy()
# convert the label value to lowercase
ground_truth['groundtruth'] = ground_truth['groundtruth'].apply(lambda x: x.strip().lower())
merge_df=pd.concat([ground_truth,prediction_df],axis=1)
merge_df

,comment,groundtruth,id,summary,reason,predict_label
0,"I was almost done with the update, the progres...",update failure,s_0,Device displayed a blank screen after update c...,The comment describes an update process that f...,update failure
1,"Scratches, peeling paint, dust in the lens... ...",poor manufacturing quality,s_1,"Scratches, peeling paint, dust in the lens. Pr...","The comment mentions scratches, peeling paint,...",poor manufacturing quality
2,"My phone has been having issues lately, freque...",device freezing,s_2,"Phone frequently freezes, becomes unresponsive...","The comment describes the phone freezing, beco...","device freezing, automatic restart, shutdown"
3,"After the update, WeChat often crashes and som...",app crashes,s_3,"After the update, WeChat often crashes and som...",The comment mentions app crashes and inability...,app crashes
4,The connectivity issues with this phone are re...,carplay connection failure,s_4,"The phone has unstable connectivity, especiall...",The comment specifically mentions connectivity...,carplay connection failure
...,...,...,...,...,...,...
95,"After the recent software update, my device ha...","automatic restart, shutdown",s_95,Device automatically shut down twice after rec...,The comment mentions the device automatically ...,"automatic restart, shutdown"
96,Another bug in the new smartphone model. It se...,activation failure,s_96,"Activation process not working properly, causi...",The comment specifically mentions an issue wit...,activation failure
97,"This phone just doesn't feel right in my hand,...",poor manufacturing quality,s_97,"The phone feels uncomfortable to use, with poo...",The comment mentions discomfort in usage and p...,poor manufacturing quality
98,"After upgrading yesterday, I've started experi...",weak signal,s_98,Experiencing pop-ups and no signal after upgra...,The comment mentions issues arising after an u...,update failure


## 6.Calculate the accuracy

In [25]:
def check_contains(row):
    return row['groundtruth'] in row['predict_label']
matches = merge_df.apply(check_contains, axis=1)
count = matches.sum()
print(f"correct: {count}")
print(f"accuracy: {count/len(merge_df)*100:.2f}%")

correct: 90
accuracy: 90.00%


### Save the result 

In [ ]:
# merge_df.to_csv('result_lab_3.csv',index=False)